<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оконные функции
**Оконные функции (window functions)** — функции, которые работают по группе строк, но при этом не сводят результат к одной строке (в отличие от `GROUP BY`).

## Основные оконные функции

| Функция        | Назначение                       |
| -------------- | -------------------------------- |
| `ROW_NUMBER()` | Порядковый номер строки в группе |
| `RANK()`       | Ранг с пропусками                |
| `DENSE_RANK()` | Ранг без пропусков               |
| `LAG()`        | Предыдущее значение              |
| `LEAD()`       | Следующее значение               |
| `SUM()`        | Накопительный итог               |
| `AVG()`        | Среднее по окну                  |

## ОКОННАЯ ФУНКЦИЯ SUM(...) OVER vs. GROUP BY

Представим таблицу `orders_log`:

| user\_uuid | order\_time | total\_uah |
| ---------- | ----------- | ---------- |
| u1         | 2024-01-10  | 300        |
| u1         | 2024-01-15  | 400        |
| u2         | 2024-01-12  | 100        |
| u2         | 2024-01-22  | 500        |


Результат `SUM(...) OVER`  

| user\_uuid | order\_time | total\_uah | cumulative\_total |
| ---------- | ----------- | ---------- | ----------------- |
| u1         | 2024-01-10  | 300        | 300               |
| u1         | 2024-01-15  | 400        | 700               |
| u2         | 2024-01-12  | 100        | 100               |
| u2         | 2024-01-22  | 500        | 600               |

Результат `GROUP BY`

| user\_uuid | total\_by\_user |
| ---------- | --------------- |
| u1         | 700             |
| u2         | 600             |

## Принципы написания оконных функций

### Синтаксис

```sql
SELECT column,
       функция(...) OVER (
        ...
       )
FROM таблица;
```


```sql
ФУНКЦИЯ(...) OVER (
    [PARTITION BY разделение_по_группам]
    [ORDER BY порядок_внутри_группы]
    [ROWS BETWEEN ... AND ...]
)
```

| Элемент            | Обязательно? | Назначение                                                                |
| ------------------ | ------------ | ------------------------------------------------------------------------- |
| `ФУНКЦИЯ(...)`     | ✅ Да         | Оконная функция (`SUM`, `ROW_NUMBER`, `LAG`, и т.д.)                      |
| `OVER (...)`       | ✅ Да         | Обязательный блок, который делает функцию оконной                         |
| `PARTITION BY ...` | ❌ Нет        | Разделение на подгруппы (например, по `user_id`)                          |
| `ORDER BY ...`     | ❌ Нет        | Определяет порядок строк в пределах каждой группы                         |
| `ROWS BETWEEN ...` | ❌ Нет        | Уточняет окно: какие строки из отсортированной группы участвуют в расчёте |

### Компоненты оконной функции

#### `ФУНКЦИЯ(...)`

Это агрегатная или оконная функция, применяемая **к окну строк** (не ко всей таблице). Например:

* `SUM()`, `AVG()`, `COUNT()`
* `ROW_NUMBER()`, `RANK()`, `LAG()`, `LEAD()`

#### `OVER (...)` — "окно", по которому будет работать функция

Ключевое отличие от обычной агрегации: **результат остаётся в каждой строке**, а не сворачивается.

##### `PARTITION BY`

* Делит данные на **группы** (разбиения).
* Каждая группа обрабатывается отдельно.

📌 Аналог `GROUP BY`, но результат **не сворачивается**, а остаётся построчно.

**Пример:**

```sql
SUM(total_uah) OVER (PARTITION BY user_uuid)
```

Считает сумму для каждого пользователя по всем его строкам.

##### `ORDER BY`

* Упорядочивает строки **внутри каждой группы (partition)**.
* Обязательно для функций, которым важен **порядок**: `ROW_NUMBER()`, `RANK()`, `LAG()`, `LEAD()`, `CUMULATIVE SUM`.

📌 Если **не указать `ORDER BY`**, то:

* Для `LAG`, `LEAD`, `ROW_NUMBER` и других **порядок будет неопределён** — результат будет некорректным.
* Для `SUM(...) OVER (PARTITION BY ...)` без `ORDER BY` — будет сумма **по всей группе**, но без накопления.

##### `ROWS BETWEEN ... AND ...` — диапазон строк

Определяет, **какой диапазон строк** внутри "окна" включить для расчёта.

**Общая форма:**

```sql
ROWS BETWEEN <начало_окна> AND <конец_окна>
```
---

**Возможные значения**

| Значение              | Что означает                             |
| --------------------- | ---------------------------------------- |
| `UNBOUNDED PRECEDING` | Самое начало окна (первая строка группы) |
| `n PRECEDING`         | `n` строк до текущей                     |
| `CURRENT ROW`         | Текущая строка                           |
| `n FOLLOWING`         | `n` строк после текущей                  |
| `UNBOUNDED FOLLOWING` | До самого конца окна                     |

---

**Примеры комбинаций**

| Синтаксис                                                  | Что делает                                                      |
| ---------------------------------------------------------- | --------------------------------------------------------------- |
| `ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW`         | Накопительная сумма от начала до текущей строки                 |
| `ROWS BETWEEN 3 PRECEDING AND CURRENT ROW`                 | Скользящее окно из 4 строк: текущая и 3 перед ней               |
| `ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING`                 | Соседи вокруг строки (3 строки: до, текущая, после)             |
| `ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING`         | От текущей строки до конца группы                               |
| `ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING` | Вся группа целиком (эквивалентна `PARTITION BY`, но с ORDER BY) |
| `ROWS BETWEEN CURRENT ROW AND CURRENT ROW`                 | Только текущая строка (по умолчанию, если ничего не указано)    |

📌 Если указано `ORDER BY`, но не указано `ROWS`, по умолчанию:

  ```sql
  ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ```

##### Пример

```sql
SELECT user_uuid,
       order_time,
       total_uah,
       SUM(total_uah) OVER (
           PARTITION BY user_uuid
           ORDER BY order_time
           ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
       ) AS cumulative_total
FROM orders_log;
```

| Элемент                  | Пояснение                                       |
| ------------------------ | ----------------------------------------------- |
| `SUM(total_uah)`         | Функция: хотим посчитать накопительную сумму    |
| `PARTITION BY user_uuid` | Для каждого пользователя отдельно               |
| `ORDER BY order_time`    | Считаем **в хронологическом порядке** заказов   |
| `ROWS BETWEEN ...`       | Накопление: с самого первого до текущего заказа |
| `AS cumulative_total`    | Название нового столбца с результатом           |


#### Типичные ошибки

| Ошибка                                                         | Что пойдет не так                                                        |
| -------------------------------------------------------------- | ------------------------------------------------------------------------ |
| **Не указали `ORDER BY` для `ROW_NUMBER()` или `LAG()`**       | Порядок будет случайным                                                  |
| **Не указали `PARTITION BY`, а он был нужен**                  | Все пользователи будут в одной группе                                    |
| **Не указали `ROWS` в агрегатных функциях (`SUM()`, `AVG()`)** | В PostgreSQL будет всё от начала до текущей строки, но лучше писать явно |

#### Как писать оконные функции правильно

1. **Определи**, по каким группам работает логика (`PARTITION BY`)
2. **Определи порядок** внутри группы (`ORDER BY`)
3. Для агрегатов — **уточни окно строк** (`ROWS BETWEEN`)
4. Убедись, что **функция применима к текущей задаче**

## Оконные функции

### Тестовая таблица `orders_log`

| user\_id | order\_date | amount |
| -------- | ----------- | ------ |
| u1       | 2024-01-01  | 100    |
| u1       | 2024-01-10  | 150    |
| u1       | 2024-01-20  | 200    |
| u2       | 2024-01-05  | 300    |
| u2       | 2024-02-01  | 100    |

### 1. `SUM(...) OVER (PARTITION BY ...)` — сумма по пользователю

```sql
SELECT user_uuid, order_time, total_uah,
      SUM(total_uah) OVER (
        PARTITION BY user_uuid
      )
FROM orders_log;
```

**Что происходит:**

* Каждой строке присваивается сумма всех `total_uah` по этому `user_uuid`.
* Порядок не важен — здесь просто агрегат без сортировки.

**Результат:**

| user\_uuid | order\_time | total_uah | sum |
| -------- | ----------- | ------ | --- |
| u1       | 2024-01-01  | 100    | 450 |
| u1       | 2024-01-10  | 150    | 450 |
| u1       | 2024-01-20  | 200    | 450 |
| u2       | 2024-01-05  | 300    | 400 |
| u2       | 2024-02-01  | 100    | 400 |


### 2. 📈 `SUM(...) OVER (PARTITION BY ... ORDER BY ...)` — кумулятивная сумма

```sql
SELECT user_uuid, order_time, total_uah,
      SUM(total_uah) OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      )
FROM orders_log;
```

**Что происходит:**

* Сначала данные разбиваются по `user_uuid`
* Затем в каждой группе сортируются по `order_time`
* Считается нарастающая сумма

**Результат:**

| user\_uuid | order\_time | total_uah | sum |
| -------- | ----------- | ------ | --- |
| u1       | 2024-01-01  | 100    | 100 |
| u1       | 2024-01-10  | 150    | 250 |
| u1       | 2024-01-20  | 200    | 450 |
| u2       | 2024-01-05  | 300    | 300 |
| u2       | 2024-02-01  | 100    | 400 |


### 3. `LAG()` и `LEAD()` — предыдущее и следующее значение

```sql
SELECT user_uuid, order_time, total_uah,
      LAG(total_uah) OVER (
          PARTITION BY user_uuid
          ORDER BY order_time
      ),
      LEAD(total_uah) OVER (
          PARTITION BY user_uuid
          ORDER BY order_time
      )
FROM orders_log;
```

**Объяснение:**

* `LAG(...)` — вернёт значение предыдущей строки внутри группы
* `LEAD(...)` — вернёт значение следующей строки внутри группы

**Результат:**

| user\_uuid | order\_time | total\_uah | lag  | lead |
| -------- | ----------- | ------ | ---- | ---- |
| u1       | 2024-01-01  | 100    | null | 150  |
| u1       | 2024-01-10  | 150    | 100  | 200  |
| u1       | 2024-01-20  | 200    | 150  | null |
| u2       | 2024-01-05  | 300    | null | 100  |
| u2       | 2024-02-01  | 100    | 300  | null |


### 4. `ROW_NUMBER()` — порядковый номер в группе

```sql
SELECT user_uuid, order_time, total_uah,
      ROW_NUMBER() OVER (
        PARTITION BY user_uuid ORDER BY order_time
      )
FROM orders_log;
```

**Что делает:**

* Пронумерует строки **внутри каждой группы по `user_uuid`**
* Порядок задаётся по дате

**Результат:**

| user\_uuid | order\_time | total\_uah | row\_number |
| -------- | ----------- | ------ | ----------- |
| u1       | 2024-01-01  | 100    | 1           |
| u1       | 2024-01-10  | 150    | 2           |
| u1       | 2024-01-20  | 200    | 3           |
| u2       | 2024-01-05  | 300    | 1           |
| u2       | 2024-02-01  | 100    | 2           |


### 5. `RANK()` и `DENSE_RANK()` — ранжирование

```sql
SELECT user_uuid, total_uah,
      RANK() OVER (
          ORDER BY total_uah DESC
      ) AS rank,
      DENSE_RANK() OVER (
          ORDER BY total_uah DESC
      ) AS dense_rank
FROM orders_log;
```

**Разница:**

* `RANK()` — пропускает ранги (1, 1, 3)
* `DENSE_RANK()` — не пропускает (1, 1, 2)

**Результат:**

| user\_uuid | total\_uah | rank | dense\_rank |
| ---------- | ---------- | ---- | ----------- |
| u2         | 300        | 1    | 1           |
| u1         | 200        | 2    | 2           |
| u1         | 200        | 2    | 2           |
| u1         | 150        | 4    | 3           |
| u2         | 100        | 5    | 4           |
| u1         | 100        | 5    | 4           |

### 6. `FIRST_VALUE()` и `LAST_VALUE()`

```sql
SELECT user_uuid, order_time, total_uah,
      FIRST_VALUE(total_uah) OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      ),
      LAST_VALUE(total_uah) OVER (
         PARTITION BY user_uuid
         ORDER BY order_time
         ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
      )
FROM orders_log;
```

**Что делает:**

* `FIRST_VALUE(...)` — первое значение по заданному `ORDER BY`
* `LAST_VALUE(...)` — последнее, но нужно указать границы окна

**Результат:**

| user\_uuid | order\_time | total\_uah | first\_value | last\_value |
| -------- | ----------- | ------ | ------------ | ----------- |
| u1       | 2024-01-01  | 100    | 100          | 200         |
| u1       | 2024-01-10  | 150    | 100          | 200         |
| u1       | 2024-01-20  | 200    | 100          | 200         |
| u2       | 2024-01-05  | 300    | 300          | 100         |
| u2       | 2024-02-01  | 100    | 300          | 100         |

## Задачи

In [15]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [16]:
DB_USER = "da_test_user"
DB_PASS = "c89M2psBdXGtBXpY3BPrTRZ2bXl6yq2i"
DB_HOST = "dpg-d1hvnobe5dus739ipbcg-a.frankfurt-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "da_test"

In [17]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

**Task 1. Кумулятивная выручка по каждому пользователю**  
Как накапливалась сумма заказов у каждого пользователя по времени?

In [19]:
query = """
SELECT user_uuid, order_time, total_uah,
  SUM(total_uah) OVER (
    PARTITION BY user_uuid
    ORDER BY order_time
  ) AS cumulative_total
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(10)

user_uuid  order_time  total_uah  cumulative_total
0   USR0000  2024-01-03    1857.51           1857.51
1   USR0000  2024-01-31    1483.19           3340.70
2   USR0000  2024-02-22    1883.10           5223.80
3   USR0000  2024-03-23     199.89           5423.69
4   USR0001  2024-02-19    2093.49           2093.49
5   USR0001  2024-03-25     496.39           2589.88
6   USR0001  2024-04-12    1088.26           3678.14
7   USR0002  2024-01-02    1458.47           1458.47
8   USR0002  2024-01-04     702.62           2161.09
9   USR0002  2024-01-06    1260.99           3422.08

**Task 2. Интервал между покупками**  
Сколько дней прошло с предыдущего заказа?

*Цель:* Выявить пользователей с длинными перерывами — возможно, они уходят.

In [21]:
query = """
SELECT
  user_uuid, order_time, total_uah,
  order_time - LAG(order_time) OVER (
    PARTITION BY user_uuid
    ORDER BY order_time
  ) as day_diff
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(15)

user_uuid  order_time  total_uah  day_diff
0    USR0000  2024-01-03    1857.51       NaN
1    USR0000  2024-01-31    1483.19      28.0
2    USR0000  2024-02-22    1883.10      22.0
3    USR0000  2024-03-23     199.89      30.0
4    USR0001  2024-02-19    2093.49       NaN
5    USR0001  2024-03-25     496.39      35.0
6    USR0001  2024-04-12    1088.26      18.0
7    USR0002  2024-01-02    1458.47       NaN
8    USR0002  2024-01-04     702.62       2.0
9    USR0002  2024-01-06    1260.99       2.0
10   USR0002  2024-01-11     238.92       5.0
11   USR0002  2024-01-12    1795.79       1.0
12   USR0002  2024-01-20     339.17       8.0
13   USR0002  2024-01-20     231.21       0.0
14   USR0002  2024-02-25     127.70      36.0

**Task 3. Порядковый номер заказа**  
Какая по счёту это покупка для пользователя?

In [23]:
query = """
SELECT user_uuid, order_time, total_uah,
  ROW_NUMBER() OVER (
    PARTITION BY user_uuid
    ORDER BY order_time
  ) AS result

FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(15)

user_uuid  order_time  total_uah  result
0    USR0000  2024-01-03    1857.51       1
1    USR0000  2024-01-31    1483.19       2
2    USR0000  2024-02-22    1883.10       3
3    USR0000  2024-03-23     199.89       4
4    USR0001  2024-02-19    2093.49       1
5    USR0001  2024-03-25     496.39       2
6    USR0001  2024-04-12    1088.26       3
7    USR0002  2024-01-02    1458.47       1
8    USR0002  2024-01-04     702.62       2
9    USR0002  2024-01-06    1260.99       3
10   USR0002  2024-01-11     238.92       4
11   USR0002  2024-01-12    1795.79       5
12   USR0002  2024-01-20     339.17       6
13   USR0002  2024-01-20     231.21       7
14   USR0002  2024-02-25     127.70       8

**Task 4. Последняя сумма покупки (для сравнения)**  
Какая была последняя покупка пользователя? Как она отличается от текущей?

In [25]:
query = """
SELECT user_uuid, order_time, total_uah,
  LAST_VALUE(total_uah) OVER (
    PARTITION BY user_uuid
    ORDER BY order_time
    ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
  )
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(15)

user_uuid  order_time  total_uah  last_value
0    USR0000  2024-01-03    1857.51      199.89
1    USR0000  2024-01-31    1483.19      199.89
2    USR0000  2024-02-22    1883.10      199.89
3    USR0000  2024-03-23     199.89      199.89
4    USR0001  2024-02-19    2093.49     1088.26
5    USR0001  2024-03-25     496.39     1088.26
6    USR0001  2024-04-12    1088.26     1088.26
7    USR0002  2024-01-02    1458.47     2931.22
8    USR0002  2024-01-04     702.62     2931.22
9    USR0002  2024-01-06    1260.99     2931.22
10   USR0002  2024-01-11     238.92     2931.22
11   USR0002  2024-01-12    1795.79     2931.22
12   USR0002  2024-01-20     339.17     2931.22
13   USR0002  2024-01-20     231.21     2931.22
14   USR0002  2024-02-25     127.70     2931.22

**Task 5. Самый крупный заказ — на каком месте он произошёл?**  
Самый большой заказ был первым, вторым или позже?

*Цель:* Понять, есть ли эффект "затухания" или наоборот — рост с доверием.

In [30]:
query = """
SELECT user_uuid, order_time, total_uah,
  RANK() OVER (
    PARTITION BY user_uuid
    ORDER BY total_uah DESC
  ),
  ROW_NUMBER() OVER (
    PARTITION BY user_uuid
    ORDER BY order_time
  )

FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(15)

user_uuid  order_time  total_uah  rank  row_number
0    USR0000  2024-02-22    1883.10     1           3
1    USR0000  2024-01-03    1857.51     2           1
2    USR0000  2024-01-31    1483.19     3           2
3    USR0000  2024-03-23     199.89     4           4
4    USR0001  2024-02-19    2093.49     1           1
5    USR0001  2024-04-12    1088.26     2           3
6    USR0001  2024-03-25     496.39     3           2
7    USR0002  2024-06-21    2931.22     1          14
8    USR0002  2024-05-01    2784.29     2          11
9    USR0002  2024-06-15    1989.58     3          13
10   USR0002  2024-04-07    1921.68     4           9
11   USR0002  2024-01-12    1795.79     5           5
12   USR0002  2024-01-02    1458.47     6           1
13   USR0002  2024-01-06    1260.99     7           3
14   USR0002  2024-06-15    1223.79     8          12

**Task 6. Первый заказ каждого пользователя**  
Какова сумма первого заказа?

In [31]:
query = """
SELECT user_uuid, order_time, total_uah,
  FIRST_VALUE(total_uah) OVER (
    PARTITION BY user_uuid
    ORDER BY order_time
  )
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(15)

user_uuid  order_time  total_uah  first_value
0    USR0000  2024-01-03    1857.51      1857.51
1    USR0000  2024-01-31    1483.19      1857.51
2    USR0000  2024-02-22    1883.10      1857.51
3    USR0000  2024-03-23     199.89      1857.51
4    USR0001  2024-02-19    2093.49      2093.49
5    USR0001  2024-03-25     496.39      2093.49
6    USR0001  2024-04-12    1088.26      2093.49
7    USR0002  2024-01-02    1458.47      1458.47
8    USR0002  2024-01-04     702.62      1458.47
9    USR0002  2024-01-06    1260.99      1458.47
10   USR0002  2024-01-11     238.92      1458.47
11   USR0002  2024-01-12    1795.79      1458.47
12   USR0002  2024-01-20     339.17      1458.47
13   USR0002  2024-01-20     231.21      1458.47
14   USR0002  2024-02-25     127.70      1458.47

**Task 7. Признак повторной покупки**  
Сделал ли пользователь хотя бы 2 заказа?

In [35]:
query = """
SELECT user_uuid, order_time, total_uah,
  CASE
    WHEN
      ROW_NUMBER() OVER (
        PARTITION BY user_uuid
        ORDER BY order_time
      ) > 1
    THEN 'R'
    ELSE 'F'
  END
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df.head(15)

user_uuid  order_time  total_uah case
0    USR0000  2024-01-03    1857.51    F
1    USR0000  2024-01-31    1483.19    R
2    USR0000  2024-02-22    1883.10    R
3    USR0000  2024-03-23     199.89    R
4    USR0001  2024-02-19    2093.49    F
5    USR0001  2024-03-25     496.39    R
6    USR0001  2024-04-12    1088.26    R
7    USR0002  2024-01-02    1458.47    F
8    USR0002  2024-01-04     702.62    R
9    USR0002  2024-01-06    1260.99    R
10   USR0002  2024-01-11     238.92    R
11   USR0002  2024-01-12    1795.79    R
12   USR0002  2024-01-20     339.17    R
13   USR0002  2024-01-20     231.21    R
14   USR0002  2024-02-25     127.70    R

#  Когортный анализ  
Когда мы анализируем поведение пользователей во времени, средние значения могут вводить в заблуждение:
*  одни пользователи только что пришли, другие — давно ушли.
*  нам нужен способ смотреть, как изменяется поведение пользователей от момента их первого действия.

## Что такое когорта?

**Когорта** — это группа пользователей, которые объединились по дате первого действия:

| Пользователь | first\_seen (дата установки) | Когорта    |
| ------------ | ---------------------------- | ---------- |
| A            | 2024‑03‑01                   | 2024‑03‑01 |
| B            | 2024‑03‑01                   | 2024‑03‑01 |
| C            | 2024‑03‑02                   | 2024‑03‑02 |

---

## Для чего нужен когортный анализ?

Когортный анализ отвечает на вопросы:

*  Когда и где мы теряем пользователей?
*  Как меняется поведение пользователей после установки?
*  Когда пользователи начинают приносить выручку?

---

## Виды когорт

| Тип когорты            | Пример                                |
| ---------------------- | ------------------------------------- |
| По дате установки      | Все, кто установил приложение в марте |
| По дате авторизации    | Все, кто вошёл в аккаунт в апреле     |
| По дате первой покупки | Все, кто впервые купил в мае          |

## Retention
**Retention Day N** — сколько пользователей из когорты вернулись на `N`-й день после установки.

### Формула

$$
\text{Retention}_{n} = \frac{\text{Число пользователей, активных на день }n}{\text{Общее число в когорте}}
$$

---

### Структура когортной таблицы

| Когорта / День → | День 0 | День 1 | День 2 | День 3 |
| ---------------- | ------ | ------ | ------ | ------ |
| 2024‑03‑01       | 100    | 40     | 25     | 10     |
| 2024‑03‑02       | 80     | 35     | 15     | 5      |

* Строка = когорта (дата установки)
* Столбец = день активности после установки

---

### Какие таблицы используются?

| Таблица             | Для чего используется               |
| ------------------- | ----------------------------------- |
| `app_sessions`      | Дата установки (когорта)            |
| `product_views_log` | Дата активности (просмотры товаров) |

### Когортный анализ выручки

* Устанавливаем когорту по дате установки (`first_seen`)
* Присоединяем покупки через `devices_users_map` и `orders_log`
* Считаем, сколько дней прошло между установкой и покупкой
* Группируем по `cohort_date` и `days_since_install`, суммируя выручку

In [42]:
query = """
SELECT
  app_sessions.first_seen::date AS cohort_date,
  (orders_log.order_time - app_sessions.first_seen)::int AS days_since_install,
  SUM(orders_log.total_uah) AS revenue

FROM app_sessions

JOIN devices_users_map
  ON app_sessions.device_code = devices_users_map.device_code
JOIN orders_log
  ON devices_users_map.user_uuid = orders_log.user_uuid

WHERE app_sessions.first_seen BETWEEN '2024-03-01' AND '2024-03-31'

GROUP BY cohort_date, days_since_install
ORDER BY cohort_date, days_since_install;
"""

df = pd.read_sql(text(query), engine)
df

cohort_date  days_since_install  revenue
0    2024-03-01                 -57  2537.92
1    2024-03-01                 -48   502.71
2    2024-03-01                 -35  2687.07
3    2024-03-01                 -27  1279.29
4    2024-03-01                 -20  2493.18
..          ...                 ...      ...
423  2024-03-31                  59   479.02
424  2024-03-31                  64  1653.87
425  2024-03-31                  75  1804.96
426  2024-03-31                  80  1435.34
427  2024-03-31                  82  2039.67

[428 rows x 3 columns]

In [38]:
# Преобразуем в когортную матрицу
cohort_matrix = df.pivot_table(
    index="cohort_date",
    columns="days_since_install",
    values="revenue",
    fill_value=0
)

cohort_matrix

days_since_install     -88      -84     -83      -82     -81      -80   \
cohort_date                                                              
2024-03-01             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-02             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-03             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-04             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-05             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-06             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-07             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-10             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-11             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-13             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-14             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-15             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-16             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-17             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-18             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-20             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-21             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-23             0.00     0.00    0.00  2935.16    0.00     0.00   
2024-03-24             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-25             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-26             0.00  2278.51    0.00     0.00    0.00     0.00   
2024-03-27             0.00  1817.92    0.00     0.00    0.00     0.00   
2024-03-28             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-29          2248.08     0.00  330.05     0.00  648.93  2742.01   
2024-03-30             0.00     0.00    0.00     0.00    0.00     0.00   
2024-03-31             0.00  1851.15    0.00     0.00    0.00     0.00   

days_since_install     -79     -78      -76      -75   ...      102      103  \
cohort_date                                            ...                     
2024-03-01             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-02             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-03             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-04             0.00    0.00     0.00     0.00  ...  2609.52   244.34   
2024-03-05             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-06             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-07             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-10             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-11             0.00    0.00     0.00     0.00  ...     0.00  3752.91   
2024-03-13             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-14             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-15             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-16             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-17             0.00    0.00     0.00  1100.52  ...     0.00     0.00   
2024-03-18             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-20             0.00  712.85  1658.02     0.00  ...     0.00     0.00   
2024-03-21             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-23          2299.49    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-24             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-25             0.00    0.00   949.09     0.00  ...     0.00     0.00   
2024-03-26             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03-27             0.00    0.00     0.00     0.00  ...     0.00     0.00   
2024-03

In [39]:
# Нормализуем по дню 0 — считаем Retention %
retention_matrix = cohort_matrix.divide(cohort_matrix[0], axis=0).round(3)

# Сбросим индекс для визуализации
retention_long = retention_matrix.reset_index().melt(
    id_vars="cohort_date",
    var_name="days_since_install",
    value_name="retention_rate"
)

fig = px.imshow(
    retention_matrix,
    labels=dict(x="День с установки", y="Дата установки", color="Retention"),
    x=retention_matrix.columns,
    y=retention_matrix.index,
    color_continuous_scale="Blues",
    text_auto=True
)

fig.update_layout(
    title="📊 Retention по когортам (по просмотрам товаров)",
    xaxis_title="День с момента установки",
    yaxis_title="Дата установки (когорта)",
    yaxis=dict(autorange="reversed")
)

fig.show()


#### **Каждая строка (Y)** — когорта пользователей, установивших приложение в один день  
Например, строка `2024‑03‑10` — это все пользователи, установившие приложение 10 марта.  

#### **Каждый столбец (X)** — день после установки

Например, `0` — день установки, `1` — следующий день, `7` — неделя спустя и т.д.

#### **Цвет ячейки** — выручка от пользователей этой когорты в конкретный день

Чем темнее цвет → тем больше выручка.

---

#### Что ты можешь узнать с такого графика:

##### ✅ **Когда когорты приносят деньги**

* Если тёмные ячейки находятся ближе к дню `0` или `1`, значит **пользователи покупают сразу** после установки.
* Если они распределены позже — значит **покупки происходят спустя некоторое время** (это может быть важно для оценки LTV).

##### ✅ **Как отличаются когорты**

* Например, когорта 31 марта явно принесла больше выручки, чем 17 марта → стоит понять, **какой канал привлечения** использовался тогда.

##### ✅ **Повторные покупки**

* Если выручка тянется в дальние дни (день 20, 30, 50…), значит пользователи **возвращаются и совершают новые заказы**.
* Если всё сосредоточено в `day 0–1`, значит продукт — **одноразовый**, и стоит задуматься об удержании.

##### ✅ **Сезонность и изменения в продукте**

* Вдруг с какой-то даты когорты стали резко приносить меньше — это может быть связано с **изменениями в UX**, **багами** или **новым каналом привлечения**.

---

#### Возможные выводы:

1. **Быстрые покупки?** — Да/Нет (по активности в первые дни).
2. **Есть ли "длинные хвосты"?** — Дни спустя 10+ → признак долгосрочной ценности.
3. **Какая когорта самая прибыльная?** — По насыщенности цвета.
4. **Нужно ли работать над удержанием?** — Если выручка резко обрывается → да.

# Легенда
* как пользователи взаимодействуют с приложением: сколько установок, сколько просматривают товары, сколько покупают;
* где теряются пользователи (на каком этапе воронки);
* какова эффективность разных каналов привлечения.

In [43]:
# сколько установок
query = """
SELECT COUNT(DISTINCT device_code) AS installs
FROM app_sessions;
"""

df = pd.read_sql(text(query), engine)
df

installs
0       500

In [44]:
query = """
SELECT COUNT(DISTINCT device_code) AS views
FROM product_views_log;
"""

df = pd.read_sql(text(query), engine)
df

views
0    500

In [45]:
query = """
SELECT COUNT(DISTINCT user_uuid) AS purchases
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df

purchases
0        300

In [47]:
query = """
SELECT user_uuid, COUNT(DISTINCT device_code) AS devices
FROM devices_users_map
GROUP BY user_uuid
HAVING COUNT(DISTINCT device_code) > 1;
"""

df = pd.read_sql(text(query), engine)
df

Empty DataFrame
Columns: [user_uuid, devices]
Index: []

In [48]:
query = """
SELECT COUNT(DISTINCT user_uuid) AS users
FROM orders_log;
"""

df = pd.read_sql(text(query), engine)
df


users
0    300

In [53]:
query = """

SELECT app_sessions.acquisition_channel,
  COUNT(DISTINCT app_sessions.device_code) AS installs,
  COUNT(DISTINCT product_views_log.device_code) AS views,
  COUNT(DISTINCT orders_log.user_uuid) AS purchases
FROM app_sessions
LEFT JOIN product_views_log
  ON app_sessions.device_code = product_views_log.device_code
LEFT JOIN devices_users_map
  ON app_sessions.device_code = devices_users_map.device_code
LEFT JOIN orders_log
  ON devices_users_map.user_uuid = orders_log.user_uuid

GROUP BY app_sessions.acquisition_channel
ORDER BY purchases DESC;
"""

df = pd.read_sql(text(query), engine)
df

acquisition_channel  installs  views  purchases
0            Facebook       134    134         89
1            Referral       132    132         73
2             Organic       111    111         70
3          Google Ads       123    123         68